In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.saving import register_keras_serializable

# === 1. Load flash flood features ===
X = pd.read_csv("dataset/flash_flood_data.csv")
X = X.drop(columns=['flash_binary'])

# === 2. Generate trust modulation labels ===
def generate_trust_score(row):
    rain = row['rainfall_intensity']
    slope = row['slope']
    drainage = row['drainage_density']
    saturation = row['soil_saturation']
    convergence = row['convergence_index']

    rain_factor = np.clip((rain - 75) / 45, 0, 1)             # More intensity → higher trust
    slope_factor = np.clip((slope - 10) / 15, 0, 1)           # Steeper terrain amplifies risk
    drainage_factor = 1 - np.clip((drainage - 1.5) / 3, 0, 1) # Lower drainage → higher risk
    saturation_factor = np.clip((saturation - 0.4) / 0.6, 0, 1) # Higher soil saturation amplifies flash likelihood
    convergence_factor = np.clip((convergence - 0.3) / 0.7, 0, 1) # More channel convergence = higher trust

    composite = (0.25 * rain_factor +
                 0.2 * slope_factor +
                 0.2 * drainage_factor +
                 0.2 * saturation_factor +
                 0.15 * convergence_factor)

    return 0.5 + composite  # Output range [0.5, 1.5]

mod_labels = X.apply(generate_trust_score, axis=1).values.astype(np.float32)

# === 3. Normalize features ===
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# === 4. Define Trust-Modulation Regression Model ===
@register_keras_serializable()
def trust_activation(x):
    return 0.5 + tf.sigmoid(x)  # Output in [0.5, 1.5]

input_layer = layers.Input(shape=(5,))
z = layers.Dense(32, activation='relu')(input_layer)
z = layers.Dense(16, activation='relu')(z)
z = layers.Dense(8, activation='relu')(z)
output = layers.Dense(1, activation=trust_activation)(z)

FlashFloodTrustNet = models.Model(inputs=input_layer, outputs=output)
FlashFloodTrustNet.compile(optimizer='adam', loss='mse')
FlashFloodTrustNet.summary()

# === 5. Train the model ===
FlashFloodTrustNet.fit(X_scaled, mod_labels, batch_size=32, epochs=12, validation_split=0.2)

# === 6. Save the model and scaler ===
FlashFloodTrustNet.save("models/FlashFloodTrustNet.h5")

import joblib
joblib.dump(scaler, "models/FlashFloodTrustScaler.pkl")

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 5)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │           192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 865 (3.38 KB)

 Trainable params: 865 (3.38 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/12
85/85 ━━━━━━━━━━━━━━━━━━━━ 6s 26ms/step - loss: 0.0835 - val_loss: 0.0051
Epoch 2/12
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0033 - val_loss: 0.0015
Epoch 3/12
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0015 - val_loss: 0.0011
Epoch 4/12
85/85 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0011 - val_loss: 7.8840e-04
Epoch 5/12
85/85 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 7.9671e-04 - val_loss: 6.0135e-04
Epoch 6/12
85/85 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 6.6548e-04 - val_loss: 5.3034e-04
Epoch 7/12
85/85 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 5.5857e-04 - val_loss: 4.8268e-04
Epoch 8/12
85/85 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 4.8649e-04 - val_loss: 4.3979e-04
Epoch 9/12
85/85 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 4.5798e-04 - val_loss: 4.2598e-04
Epoch 10/12
85/85 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 4.2706e-04 - val_loss: 3.9810e-04
Epoch 11/12
85/85 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 3.9529e-04 - val_loss: 3.7554e-04
Epoch 12/12
8

['models/FlashFloodTrustScaler.pkl']